In [110]:
# !pip install openpyxl

In [113]:
# ============================================================
# 🧭 Y06_안전비상벨_TM 주소 표준화 전처리 (최종 정리)
# - ID 정리 (공백/탭 제거)
# - 중복 주소 컬럼 통합
# - 시/구/동/도로명주소 분리
# - 표 출력 및 CSV 저장
# ============================================================

import pandas as pd
import re
from IPython.display import display

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y06_안전비상벨_TM.csv"
output_file = "Y06_안전비상벨_TM_전처리.csv"

# ===== 1) 파일 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 원본 데이터 로드 완료: {df.shape}")

✅ 원본 데이터 로드 완료: (1000, 39)


In [114]:
# ===== 2) 앞에 붙은 ID 값 정리 (공백/탭 허용 버전) =====
df['_OBJT_ID'] = df.iloc[:, 0].apply(
    lambda x: re.match(r"^\s*(\d+)", str(x)).group(1) if pd.notna(x) and re.match(r"^\s*(\d+)", str(x)) else None
)

df.iloc[:, 0] = df.iloc[:, 0].apply(
    lambda x: re.sub(r"^\s*\d+\s*", "", str(x)) if pd.notna(x) else x
)

# ID를 인덱스로 설정
df = df.set_index("_OBJT_ID")
df.index.name = "ID"

# ===== 3) 주소 관련 컬럼 통합 =====
address_cols = [col for col in df.columns if any(keyword in col for keyword in ["주소", "ADRES", "addr", "Addr"])]
print(f"📍 주소 관련 컬럼 후보: {address_cols}")

def pick_address(row):
    for col in address_cols:
        val = row.get(col)
        if pd.notna(val) and str(val).strip() != "":
            return str(val).strip()
    return None

df["표준주소"] = df.apply(pick_address, axis=1)

# ===== 4) 주소 파싱 (시/구/동/도로명주소) =====
def split_address(addr):
    if pd.isna(addr):
        return None, None, None, None
    parts = re.split(r"\s+", str(addr).strip())
    si   = parts[0] if len(parts) > 0 else None
    gu   = parts[1] if len(parts) > 1 and re.search(r"(시|군|구)$", parts[1]) else None
    dong = parts[2] if len(parts) > 2 else None
    road = " ".join(parts[3:]) if len(parts) > 3 else None
    return si, gu, dong, road

df[["시", "구", "동", "도로명주소"]] = df["표준주소"].apply(lambda x: pd.Series(split_address(x)))

# ===== 5) 중복 주소 컬럼 정리 =====
df = df.drop(columns=[col for col in address_cols if col != "표준주소"], errors="ignore")

# ===== 6) CSV 저장 =====
df.to_csv(output_file, encoding="utf-8-sig")
print(f"💾 주소 표준화 완료 CSV 저장: {output_file}")

# ===== 7) 결과 확인 =====
print("🧾 결과 컬럼 목록:")
print(df.columns.tolist())
display(df.head(20))

📍 주소 관련 컬럼 후보: ['RN_ADRES', 'ADRES', '입력주소', '주소구분', '표준신주소', '표준구주소']
💾 주소 표준화 완료 CSV 저장: Y06_안전비상벨_TM_전처리.csv
🧾 결과 컬럼 목록:
['FCLTY_TY', 'MNG_INST', 'INS_PURPOS', 'INS_TYPE', 'INS_DETAIL', 'LAT', 'LON', 'LNK_TYPE', 'FLAG_POL_L', 'FLAG_SEC_L', 'FLAG_MNG_L', 'ADDITION', 'INS_YEAR', 'LAST_INSPD', 'LAST_INSPT', 'MNG_TEL', 'FLAG_SERVI', 'CTPRVN_CD', 'SGG_CD', 'EMD_CD', 'X', 'Y', 'DATA_TY', 'X.1', 'Y.1', 'CLSS', 'PNU', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명', '표준주소', '시', '구', '동', '도로명주소']


,FCLTY_TY,MNG_INST,INS_PURPOS,INS_TYPE,INS_DETAIL,LAT,LON,LNK_TYPE,FLAG_POL_L,FLAG_SEC_L,...,우편번호,행정동코드,행정동명,법정동코드,법정동명,표준주소,시,구,동,도로명주소
ID,,,,,,,,,,,,,,,,,,,,,
,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구 서시정),37.591042,126.966257,양방향,Y,N,...,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동,-,-,None,None,None
,비상벨,종로구청,방범용,공원,인왕산도시자연공원,37.591042,126.966257,양방향,N,N,...,NaN,1.111052e+09,청운효자동,1.111010e+09,청운동,-,-,None,None,None
,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(청운지구),37.589514,126.965510,양방향,Y,N,...,NaN,NaN,NaN,1.111010e+09,청운동,-,-,None,None,None
,비상벨,종로구청,약자보호,화장실,창의문화장실,37.591324,126.971784,양방향,Y,N,...,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동,-,-,None,None,None
,비상벨,종로구청,약자보호,화장실,창의문화장실,37.591324,126.971784,양방향,Y,N,...,3048.0,1.111052e+09,청운효자동,1.111010e+09,청운동,-,-,None,None,None
,비상벨,종로구청,약자보호,화장실,무궁화동산,37.584386,126.972402,단방향,Y,N,...,NaN,1.111052e+09,청운효자동,1.111010e+09,궁정동,-,-,None,None,None
,비상벨,종로구청 청소행정과,방범용,화장실,누상동체육시설 공중화장실,37.580046,126.964202,미연계,N,N,...,3038.0,1.111052e+09,청운효자동,1.111011e+09,누상동,-,-,None,None,None
,비상벨,종로구청,약자보호,화장실,인왕산도시자연공원(누상지구),37.578974,126.963164,양방향,Y,N,...,NaN,1.111052e+09,청운효자동,1.111011e+09,누상동,-,-,None,None,None
,비상벨,종로구청,방범용,공원,수성동계곡,37.582070,126.963764,양방향,N,N,...,NaN,NaN,NaN,1.111011e+09,옥인동,-,-,None,None,None
